In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# --- Configuration ---
Lx, Ly = 50, 50       
Nx, Ny = 100, 100      # Lower res for faster rendering in notebook
dt = 0.001            # Small dt for stability
n_seconds = 2.0       # Total physical time to simulate
save_every = 10     # Don't save every dt (too much data), save every nth step

# Derived parameters
dx = Lx / Nx
dy = Ly / Ny
total_steps = int(n_seconds / dt)
n_frames = total_steps // save_every

# --- Grid & Initial Conditions ---
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

np.random.seed(42)
u = np.random.uniform(-0.1, 0.1, (Nx, Ny))

# Storage for PySINDy (Time, X, Y)
# We pre-allocate to avoid memory fragmentation
u_data = np.zeros((n_frames, Nx, Ny))
t_data = np.zeros(n_frames)

# --- Derivatives ---
def get_laplacian(f):
    return (np.roll(f, -1, axis=0) + np.roll(f, 1, axis=0) +
            np.roll(f, -1, axis=1) + np.roll(f, 1, axis=1) - 4*f) / (dx**2)

def get_gradients(f):
    gx = (np.roll(f, -1, axis=0) - np.roll(f, 1, axis=0)) / (2*dx)
    gy = (np.roll(f, -1, axis=1) - np.roll(f, 1, axis=1)) / (2*dy)
    return gx, gy

def compute_rhs(u):
    lap = get_laplacian(u)
    lap_lap = get_laplacian(lap)
    gx, gy = get_gradients(u)
    # KS Equation: -Laplacian - Biharmonic - 0.5*|Grad|^2
    return -lap - lap_lap - 0.5*(gx**2 + gy**2)

# --- Run Simulation ---
print(f"Simulating {total_steps} steps...")
current_u = u.copy()
frame_idx = 0

for step in range(total_steps):
    # Integration
    rhs = compute_rhs(current_u)
    current_u += dt * rhs
    current_u = np.nan_to_num(current_u) # Safety

    # Storage
    if step % save_every == 0:
        u_data[frame_idx, :, :] = current_u.copy()
        t_data[frame_idx] = step * dt
        frame_idx += 1

print("Simulation complete.")

# --- Visualize in Notebook ---
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(u_data[0], cmap='inferno', origin='lower', extent=[0, Lx, 0, Ly], animated=True)
ax.set_title("KS Equation Simulation")
plt.colorbar(im, label="u(x,y)")

def animate(i):
    im.set_array(u_data[i])
    # Auto-scale color to see the instability grow
    im.set_clim(np.min(u_data[i]), np.max(u_data[i]))
    return im,

ani = animation.FuncAnimation(fig, animate, frames=len(u_data), interval=100, blit=True)
plt.close() # Prevents duplicate static plot

# This line displays the video player in Jupyter
HTML(ani.to_jshtml())

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso
from itertools import combinations_with_replacement
import matplotlib.pyplot as plt

# --- Extract data from simulation ---
# u_data shape: (n_frames, Nx, Ny)
# Reshape for SINDy: (n_samples, n_spatial_points)
n_frames, Nx, Ny = u_data.shape
n_spatial = Nx * Ny

# Flatten spatial dimensions
u_flat = u_data.reshape(n_frames, n_spatial)  # (200, 10000)

# --- Compute temporal derivative ---
# ∂u/∂t via finite differences
dt_data = np.diff(u_flat, axis=0) / (dt * save_every)  # (199, 10000)
u_flat = u_flat[:-1]  # Trim to match (199, 10000)

# --- Compute spatial derivatives for each point ---
# We need: ∇u, ∇²u, ∇⁴u, |∇u|²
# These are computed on the 2D grid before flattening

def compute_pde_library(u_data, dx, dy):
    """
    Compute a library of spatial derivatives and nonlinear terms.
    Returns library matrix where each column is a candidate term.
    """
    n_frames, Nx, Ny = u_data.shape
    n_spatial = Nx * Ny
    
    library_terms = []
    term_names = []
    
    for frame in range(n_frames):
        u = u_data[frame]
        
        # 1st derivatives
        ux = (np.roll(u, -1, axis=0) - np.roll(u, 1, axis=0)) / (2*dx)
        uy = (np.roll(u, -1, axis=1) - np.roll(u, 1, axis=1)) / (2*dy)
        
        # Laplacian
        lap = (np.roll(u, -1, axis=0) + np.roll(u, 1, axis=0) +
               np.roll(u, -1, axis=1) + np.roll(u, 1, axis=1) - 4*u) / (dx**2)
        
        # Biharmonic (Laplacian of Laplacian)
        lap_lap = (np.roll(lap, -1, axis=0) + np.roll(lap, 1, axis=0) +
                   np.roll(lap, -1, axis=1) + np.roll(lap, 1, axis=1) - 4*lap) / (dx**2)
        
        # Nonlinear term: |∇u|²
        grad_squared = ux**2 + uy**2
        
        # Store flattened terms
        if frame == 0:
            # Build library column names on first frame
            library_terms.append(lap.flatten())
            term_names.append('∇²u')
            library_terms.append(lap_lap.flatten())
            term_names.append('∇⁴u')
            library_terms.append(grad_squared.flatten())
            term_names.append('|∇u|²')
        else:
            library_terms[0] = np.vstack([library_terms[0], lap.flatten()])
            library_terms[1] = np.vstack([library_terms[1], lap_lap.flatten()])
            library_terms[2] = np.vstack([library_terms[2], grad_squared.flatten()])
    
    # Convert to proper format: (n_samples, n_features)
    library_matrix = np.hstack([term.reshape(-1, 1) if len(term.shape) == 1 
                                 else term 
                                 for term in library_terms])
    
    return library_matrix, term_names

# Simpler approach: compute library for each spatial point independently
print("Computing PDE library...")

# Collect all derivatives across all frames and spatial points
library_list = []

for frame in range(n_frames - 1):
    u = u_data[frame]
    
    # Compute derivatives on 2D grid
    ux = (np.roll(u, -1, axis=0) - np.roll(u, 1, axis=0)) / (2*dx)
    uy = (np.roll(u, -1, axis=1) - np.roll(u, 1, axis=1)) / (2*dy)
    lap = (np.roll(u, -1, axis=0) + np.roll(u, 1, axis=0) +
           np.roll(u, -1, axis=1) + np.roll(u, 1, axis=1) - 4*u) / (dx**2)
    lap_lap = (np.roll(lap, -1, axis=0) + np.roll(lap, 1, axis=0) +
               np.roll(lap, -1, axis=1) + np.roll(lap, 1, axis=1) - 4*lap) / (dx**2)
    grad_squared = ux**2 + uy**2
    
    # Flatten and stack
    lib = np.column_stack([
        lap.flatten(),
        lap_lap.flatten(),
        grad_squared.flatten()
    ])
    library_list.append(lib)

library = np.vstack(library_list)  # (samples, 3 features)
print(f"Library shape: {library.shape}")
print(f"Target (du/dt) shape: {dt_data.flatten().shape}")

# --- Sparse Regression (SINDy) ---
print("\nPerforming sparse regression...")
target = dt_data.flatten()  # Flatten all du/dt values

# Use Lasso with low alpha for sparse identification
model = Lasso(alpha=1e-5, max_iter=10000, fit_intercept=False)
model.fit(library, target)

# --- Results ---
coefficients = model.coef_
term_names = ['∇²u', '∇⁴u', '|∇u|²']

print("\n" + "="*60)
print("DISCOVERED PDE COEFFICIENTS:")
print("="*60)
print("du/dt = c₁·∇²u + c₂·∇⁴u + c₃·|∇u|²\n")

for name, coeff in zip(term_names, coefficients):
    if abs(coeff) > 1e-6:
        print(f"{name:15s}: {coeff:10.6f}")
    else:
        print(f"{name:15s}: {coeff:10.6f} (negligible)")

print("\n" + "="*60)
print("ORIGINAL PDE COEFFICIENTS:")
print("="*60)
print("du/dt = -1.000000·∇²u + -1.000000·∇⁴u + -0.500000·|∇u|²\n")

print("="*60)
print("COMPARISON:")
print("="*60)
original = np.array([-1.0, -1.0, -0.5])
discovered = coefficients

for i, (name, orig, disc) in enumerate(zip(term_names, original, discovered)):
    error = abs(orig - disc) / abs(orig) * 100
    print(f"{name:15s}: Original={orig:10.6f}, Discovered={disc:10.6f}, Error={error:6.2f}%")

print(f"\nModel R² score: {model.score(library, target):.6f}")

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt

# --- Weak-SINDy: Integral Formulation ---
# Instead of pointwise matching: du/dt = Θ(u)·c
# We match: ∫(du/dt)·φ dxdy = ∫Θ(u)·c·φ dxdy for multiple test functions φ

print("="*60)
print("WEAK-SINDy IMPLEMENTATION (Integral Formulation)")
print("="*60)

# --- Define test functions ---
# Use Fourier basis or polynomial basis as test functions
def create_test_functions(Nx, Ny, n_test_modes=20):
    """
    Create orthogonal test functions using Fourier basis.
    Returns (n_test, Nx, Ny) array
    """
    test_functions = []
    mode_idx = 0
    
    # Fourier modes: cos(πkx/Lx)·cos(πly/Ly) and sin variants
    for kx in range(int(np.sqrt(n_test_modes)) + 1):
        for ky in range(int(np.sqrt(n_test_modes)) + 1):
            if mode_idx >= n_test_modes:
                break
            x_grid = np.arange(Nx)
            y_grid = np.arange(Ny)
            
            # Cosine-cosine mode
            test_fn = np.outer(np.cos(np.pi * kx * x_grid / Nx), 
                               np.cos(np.pi * ky * y_grid / Ny))
            test_functions.append(test_fn / np.linalg.norm(test_fn))
            mode_idx += 1
    
    return np.array(test_functions[:n_test_modes])

# Create test functions
n_test_modes = 16  # Number of test functions
test_fns = create_test_functions(Nx, Ny, n_test_modes)
print(f"Created {n_test_modes} test functions")

# --- Compute weak residuals ---
# For each test function φ, compute:
# - ∫(du/dt)·φ dxdy  (RHS)
# - ∫Θ(u)·φ dxdy     (Library matrix, one column per term)

print("\nComputing weak residuals...")

weak_library = []  # Will be (n_test_modes, 3)
weak_target = []   # Will be (n_test_modes,)

for test_fn in test_fns:
    row_lib = []
    
    # Compute integral of temporal derivative with test function
    integral_dudt = 0.0
    integral_lap = 0.0
    integral_lap_lap = 0.0
    integral_grad_sq = 0.0
    
    for frame in range(n_frames - 1):
        u = u_data[frame]
        u_next = u_data[frame + 1]
        
        # Temporal derivative
        dudt = (u_next - u) / (dt * save_every)
        
        # Spatial derivatives
        ux = (np.roll(u, -1, axis=0) - np.roll(u, 1, axis=0)) / (2*dx)
        uy = (np.roll(u, -1, axis=1) - np.roll(u, 1, axis=1)) / (2*dy)
        
        lap = (np.roll(u, -1, axis=0) + np.roll(u, 1, axis=0) +
               np.roll(u, -1, axis=1) + np.roll(u, 1, axis=1) - 4*u) / (dx**2)
        
        lap_lap = (np.roll(lap, -1, axis=0) + np.roll(lap, 1, axis=0) +
                   np.roll(lap, -1, axis=1) + np.roll(lap, 1, axis=1) - 4*lap) / (dx**2)
        
        grad_squared = ux**2 + uy**2
        
        # Integrate over spatial domain (using simple sum as quadrature)
        integral_dudt += np.sum(dudt * test_fn) * dx * dy
        integral_lap += np.sum(lap * test_fn) * dx * dy
        integral_lap_lap += np.sum(lap_lap * test_fn) * dx * dy
        integral_grad_sq += np.sum(grad_squared * test_fn) * dx * dy
    
    # Average over frames
    integral_dudt /= (n_frames - 1)
    integral_lap /= (n_frames - 1)
    integral_lap_lap /= (n_frames - 1)
    integral_grad_sq /= (n_frames - 1)
    
    weak_target.append(integral_dudt)
    row_lib.append([integral_lap, integral_lap_lap, integral_grad_sq])
    
    weak_library.append(row_lib[0])

weak_library = np.array(weak_library)  # (n_test_modes, 3)
weak_target = np.array(weak_target)    # (n_test_modes,)

print(f"Weak library shape: {weak_library.shape}")
print(f"Weak target shape: {weak_target.shape}")

# --- Sparse regression on weak residuals ---
print("\nPerforming sparse regression on weak residuals...")

model_weak = Lasso(alpha=1e-6, max_iter=10000, fit_intercept=False)
model_weak.fit(weak_library, weak_target)

# --- Results ---
coefficients_weak = model_weak.coef_
term_names = ['∇²u', '∇⁴u', '|∇u|²']

print("\n" + "="*60)
print("WEAK-SINDy DISCOVERED COEFFICIENTS:")
print("="*60)
print("du/dt = c₁·∇²u + c₂·∇⁴u + c₃·|∇u|²\n")

for name, coeff in zip(term_names, coefficients_weak):
    if abs(coeff) > 1e-6:
        print(f"{name:15s}: {coeff:10.6f}")
    else:
        print(f"{name:15s}: {coeff:10.6f} (negligible)")

print("\n" + "="*60)
print("ORIGINAL PDE COEFFICIENTS:")
print("="*60)
print("du/dt = -1.000000·∇²u + -1.000000·∇⁴u + -0.500000·|∇u|²\n")

print("="*60)
print("COMPARISON: Regular SINDy vs Weak-SINDy:")
print("="*60)

original = np.array([-1.0, -1.0, -0.5])

print(f"\n{'Term':<15} {'Original':<12} {'Regular':<12} {'Weak':<12} {'Error(R)':<10} {'Error(W)':<10}")
print("-"*70)

for i, name in enumerate(term_names):
    orig = original[i]
    disc_reg = coefficients[i]
    disc_weak = coefficients_weak[i]
    error_reg = abs(orig - disc_reg) / abs(orig) * 100
    error_weak = abs(orig - disc_weak) / abs(orig) * 100
    
    print(f"{name:<15} {orig:<12.6f} {disc_reg:<12.6f} {disc_weak:<12.6f} {error_reg:<10.2f} {error_weak:<10.2f}")

print(f"\nRegular SINDy R² score: {model.score(library, target):.6f}")
print(f"Weak-SINDy R² score:    {model_weak.score(weak_library, weak_target):.6f}")

# --- Visualization ---
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot comparison
x_pos = np.arange(len(term_names))
width = 0.25

axes[0].bar(x_pos - width, original, width, label='Original', color='black')
axes[0].bar(x_pos, coefficients, width, label='Regular SINDy', color='steelblue')
axes[0].bar(x_pos + width, coefficients_weak, width, label='Weak-SINDy', color='coral')

axes[0].set_ylabel('Coefficient Value')
axes[0].set_title('Coefficient Comparison')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(term_names)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error comparison
errors_reg = np.abs(original - coefficients) / np.abs(original) * 100
errors_weak = np.abs(original - coefficients_weak) / np.abs(original) * 100

axes[1].bar(x_pos - width/2, errors_reg, width, label='Regular SINDy', color='steelblue')
axes[1].bar(x_pos + width/2, errors_weak, width, label='Weak-SINDy', color='coral')

axes[1].set_ylabel('Relative Error (%)')
axes[1].set_title('Relative Error from Ground Truth')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(term_names)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('sindy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved as 'sindy_comparison.png'")